# Orbits in cartesian coordinates

Here we will solve the motion of orbits of two planets under the influence of gravity using cartesian coordinates. Normally the problem is solved in the CoM frame using polar coordinates, but here we will solve it in cartesian coordinates.

$\begin{align}
  m_1 \mathbf{\ddot r_1} = \frac{G m_1 m_2}{|\mathbf{r_2 - r_1}|^{3}} (\mathbf{r_2 - r_1}) 
  \hspace{3em} \& \hspace{3em}   
  m_2 \mathbf{\ddot r_2} = \frac{G m_1 m_2}{|\mathbf{r_1 - r_2}|^{3}} (\mathbf{r_1 - r_2})
\end{align}$  

In our case we will simplify this by confining the motion to a plane. For two particles, conservation of angular momentum tells us that this is actually no simplification, the particles motion will be constrained to a plane. We will choose corrdinates with the z-axis perpendicular to our plane so the paricles have the same z coordinate. This leads to $4$ $2^{nd}$ order differential equations,

$\begin{align}
  \left(\begin{array}{c} \ddot x_1(t) \\ \ddot y_1(t) \end{array} \right) 
       = \frac{G m_2}{|\mathbf{r_2 - r_1}|^{3}} \left(\begin{array}{c} x_2(t) - x_1(t) \\ y_2(t) - y_1(t) \end{array} \right) 
     \hspace{3em} \& \hspace{3em} 
    \left(\begin{array}{c} \ddot x_2(t) \\ \ddot y_2(t) \end{array} \right) 
       = \frac{G m_1}{|\mathbf{r_1 - r_2}|^{3}} \left(\begin{array}{c} x_1(t) - x_2(t) \\ y_1(t) - y_2(t) \end{array} \right) 
\end{align}$

We can introduce, $v_{x1}$ $v_{y1}$ $v_{x2}$ $v_{y2}$ the velocities of our particles. This allows us to rewrite our $4$ $2^{nd}$ order differential equations as $8$ $1^{st}$ order equations,

$\begin{align}
  \left(\begin{array}{c} \dot v_{x1}(t) \\ \dot v_{y1}(t) \end{array} \right) 
       = \frac{G m_2}{|\mathbf{r_2 - r_1}|^{3}} \left(\begin{array}{c} x_2(t) - x_1(t) \\ y_2(t) - y_1(t) \end{array} \right) 
   \hspace{1em} \& \hspace{1em}    
   \left(\begin{array}{c} v_{x1}(t) \\ v_{y1}(t) \end{array} \right) 
   = \left(\begin{array}{c} \dot x_1(t) \\ \dot y_1(t) \end{array} \right)
\end{align}$

$\begin{align}
\left(\begin{array}{c} \dot v_{x2}(t) \\ \dot v_{y2}(t) \end{array} \right) 
       = \frac{G m_1}{|\mathbf{r_1 - r_2}|^{3}} \left(\begin{array}{c} x_1(t) - x_2(t) \\ y_1(t) - y_2(t) \end{array} \right) 
     \hspace{1em} \& \hspace{1em}
       \left(\begin{array}{c} v_{x2}(t) \\ v_{y2}(t) \end{array} \right) 
       = \left(\begin{array}{c} \dot x_2(t) \\ \dot y_2(t) \end{array} \right)
\end{align}$      

## Packages, classes, and definitions

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

from matplotlib import animation, rc
from IPython.display import HTML

In [ ]:
# Change the common font size
font_size = 14
plt.rcParams.update({'font.size': font_size})

In [ ]:
class Orbit():
    """
    Orbit class solves for the orbit of a two body system with the only interaction being gravity.
     
    Parameters
    ----------
    G : Float
        The gravitational constant
    m1: Float
        The mass of the first planet
    m2: Float 
        The mass of the second planet

    Methods
    -------
    dy_dt(t, z)
        Returns the right side of the differential equation in vector z, 
        given time t and the corresponding value of z.
    
    solve_ode
        Uses solve_ivp to solve the differential equation dy_dt passed to it.
        
    solve_ode_Leapfrog
        Uses the Leapfrog method to solve the differential equation dy_dt passed to it.
        
    energy
        Returns the energy of our system as a function of time.
    """
    def __init__(self, G=1., m1=1., m2=1.):
        self.G = G
        self.m1 = m1
        self.m2 = m2
        
    def dy_dt(self, t, z):
        """
        This function returns the right-hand side of the diffeq: 
        [dphi/dt d^2phi/dt^2]
        
        Parameters
        ----------
        z : float
            A 2-component vector with z[0] = x1(t) and z[1] = dx1(t)/dt
                                      z[2] = x2(t) and z[3] = dx2(t)/dt
                                      z[4] = y1(t) and z[5] = dy1(t)/dt
                                      z[6] = y2(t) and z[7] = dy2(t)/dt
        t : float
            time 
            
        Returns
        -------
        """
        x1, x1_dot, x2, x2_dot, y1, y1_dot, y2, y2_dot = z
        r = np.sqrt((x1 - x2)**2 + (y1 - y2)**2)
        
        return [ \
                x1_dot, self.m2 * self.G * (x2 - x1) / r**3, \
                x2_dot, self.m1 * self.G * (x1 - x2) / r**3, \
                y1_dot, self.m2 * self.G * (y2 - y1) / r**3, \
                y2_dot, self.m1 * self.G * (y1 - y2) / r**3, \
               ]
    
    def solve_ode(self, t_pts, z_0,
                  method='RK23',
                  abserr=1.0e-8, relerr=1.0e-8):
        """
        Solve the ODE given initial conditions.
        Use solve_ivp with the option of specifying the method.
        Specify smaller abserr and relerr to get more precision.
        """
        
        solution = solve_ivp(self.dy_dt, (t_pts[0], t_pts[-1]), 
                             z_0, t_eval=t_pts, method=method, 
                             atol=abserr, rtol=relerr)
        
        x1, x1_dot, x2, x2_dot, y1, y1_dot, y2, y2_dot = solution.y
        
        return x1, x1_dot, x2, x2_dot, y1, y1_dot, y2, y2_dot
    
    def solve_ode_Leapfrog(self, t_pts, z_0):
        """
        Solve the ODE given initial conditions with the Leapfrog method.
        """
        delta_t = t_pts[1] - t_pts[0]
        
        # initialize the arrays for positions and velocities with zeros
        num_t_pts = len(t_pts)
        
        x1 = np.zeros(num_t_pts)
        x2 = np.zeros(num_t_pts)
        y1 = np.zeros(num_t_pts)
        y2 = np.zeros(num_t_pts)
        x1_dot_half = np.zeros(num_t_pts)
        x2_dot_half = np.zeros(num_t_pts)
        y1_dot_half = np.zeros(num_t_pts)
        y2_dot_half = np.zeros(num_t_pts)
        x1_dot = np.zeros(num_t_pts)
        x2_dot = np.zeros(num_t_pts)
        y1_dot = np.zeros(num_t_pts)
        y2_dot = np.zeros(num_t_pts)
        
        # initial conditions
        x1[0] = z_0[0]
        x1_dot[0] = z_0[1]
        x2[0] = z_0[2]
        x2_dot[0] = z_0[3]
        y1[0] = z_0[4]
        y1_dot[0] = z_0[5]
        y2[0] = z_0[6]
        y2_dot[0] = z_0[7]
        
        # step through the differential equation
        for i in np.arange(num_t_pts - 1):
            t = t_pts[i]
            z = [x1[i], x1_dot[i], x2[i], x2_dot[i], y1[i], y1_dot[i], y2[i], y2_dot[i]]
            x1_dot_half[i] = x1_dot[i] + self.dy_dt(t, z)[1] * delta_t/2.
            x2_dot_half[i] = x2_dot[i] + self.dy_dt(t, z)[3] * delta_t/2.
            y1_dot_half[i] = y1_dot[i] + self.dy_dt(t, z)[5] * delta_t/2.
            y2_dot_half[i] = y2_dot[i] + self.dy_dt(t, z)[7] * delta_t/2.
            
            x1[i+1] = x1[i] + x1_dot_half[i]*delta_t
            x2[i+1] = x2[i] + x2_dot_half[i]*delta_t
            y1[i+1] = y1[i] + y1_dot_half[i]*delta_t
            y2[i+1] = y2[i] + y2_dot_half[i]*delta_t
            
            z = [x1[i+1], x1_dot[i], x2[i+1], x2_dot[i], y1[i+1], y1_dot[i], y2[i+1], y2_dot[i]]
            
            x1_dot[i+1] = x1_dot_half[i] + self.dy_dt(t, z)[1] * delta_t/2.
            x2_dot[i+1] = x2_dot_half[i] + self.dy_dt(t, z)[3] * delta_t/2.
            y1_dot[i+1] = y1_dot_half[i] + self.dy_dt(t, z)[5] * delta_t/2.
            y2_dot[i+1] = y2_dot_half[i] + self.dy_dt(t, z)[7] * delta_t/2.

        return x1, x1_dot, x2, x2_dot, y1, y1_dot, y2, y2_dot  
        
    
    def energy(self, t_pts, x1, x1_dot, x2, x2_dot, y1, y1_dot, y2, y2_dot):
        """Evaluate the energy as a function of time"""
        return (self.m1/2.) * ((x1_dot)**2 + (y1_dot)**2) \
                + (self.m2/2.) * ((x2_dot)**2 + (y2_dot)**2) \
                - (self.G * self.m1 * self.m2) / np.sqrt((x1-x2)**2+(y1-y2)**2)

In [ ]:
def plot_y_vs_x(x, y, axis_labels=None, label=None, title=None, 
                color=None, linestyle=None, semilogy=False, loglog=False,
                ax=None):
    """
    Generic plotting function: return a figure axis with a plot of y vs. x,
    with line color and style, title, axis labels, and line label
    """
    if ax is None:        # if the axis object doesn't exist, make one
        ax = plt.gca()

    if (semilogy):
        line, = ax.semilogy(x, y, label=label, 
                            color=color, linestyle=linestyle)
    elif (loglog):
        line, = ax.loglog(x, y, label=label, 
                          color=color, linestyle=linestyle)
    else:
        line, = ax.plot(x, y, label=label, 
                    color=color, linestyle=linestyle)

    if label is not None:    # if a label if passed, show the legend
        ax.legend()
    if title is not None:    # set a title if one if passed
        ax.set_title(title)
    if axis_labels is not None:  # set x-axis and y-axis labels if passed  
        ax.set_xlabel(axis_labels[0])
        ax.set_ylabel(axis_labels[1])

    return ax, line

In [ ]:
def start_stop_indices(t_pts, plot_start, plot_stop):
    start_index = (np.fabs(t_pts-plot_start)).argmin()  # index in t_pts array 
    stop_index = (np.fabs(t_pts-plot_stop)).argmin()  # index in t_pts array 
    return start_index, stop_index

## Orbit with equal masses

In [ ]:
G=1. 
m1=1.
m2=1.

# Common plotting time (generate the full time then use slices)
t_start = 0.
t_end = 100.
delta_t = 0.01

t_pts = np.arange(t_start, t_end+delta_t, delta_t)  

o1 = Orbit(G=G, m1=m1, m2=m2)

In order to have orbits that stay fixed at the origin we must impose the condition that the center of mass is at the origin and it's velocity is zero. To do this, we only allow that the inital conditions of one mass to be specified. 

In [ ]:
#Set these manually
x1_0 = 1.
x1_dot_0 = 0.
y1_0 = 0.
y1_dot_0 = 0.4

In [ ]:
def positionm2(x10, x1dot0, y10, y1dot0, m_1, m_2):
    x20 = (-m_1/m_2)*x10
    x2dot0 = (-m_1/m_2)*x1dot0
    y20 = (-m_1/m_2)*y10
    y2dot0 = (-m_1/m_2)*y1dot0
    return x20, x2dot0, y20, y2dot0

In [ ]:
# Calculated inital conditions
x2_0, x2_dot_0, y2_0, y2_dot_0 = positionm2(x1_0, x1_dot_0, y1_0, y1_dot_0, m1, m2)

# Calculate the trajectories
z_0 = [x1_0, x1_dot_0, x2_0, x2_dot_0, y1_0, y1_dot_0, y2_0, y2_dot_0]
x1, x1_dot, x2, x2_dot, y1, y1_dot, y2, y2_dot = o1.solve_ode(t_pts, z_0)

### Plots

In [ ]:
fig = plt.figure(figsize=(10,6))
ax_a = fig.add_subplot(1,1,1)

overall_title = 'Two Body Orbit:  ' + \
                rf'  $x_1(0) = {x1_0:.1f},$' + \
                rf'  $\dotx_1(0) = {x1_dot_0:.1f},$' + \
                rf'  $y_1(0) = {y1_0:.1f},$' + \
                rf'  $\doty_1(0) = {y1_dot_0:.1f},$' + \
                '\n' + \
                rf'  $x_2(0) = {x2_0:.1f},$' + \
                rf'  $\dotx_2(0) = {x2_dot_0:.1f},$' + \
                rf'  $y_2(0) = {y2_0:.1f},$' + \
                rf'  $\doty_2(0) = {y2_dot_0:.1f},$' + \
                '\n' + \
                rf'  $G = {G:.1f}$' + \
                rf'  $m_1 = {m1:.1f},$' + \
                rf'  $m_2 = {m2:.1f},$'
fig.suptitle(overall_title, va='baseline')

start, stop = start_stop_indices(t_pts, t_start, t_end)    
plot_y_vs_x(x1[start : stop], y1[start : stop], 
            axis_labels=(r'$x$', r'$y$'),
            color='blue',
            label=r'$m_1$',  
            ax=ax_a) 
plot_y_vs_x(x2[start : stop], y2[start : stop], 
            axis_labels=(r'$x$', r'$y$'),
            color='red',
            label=r'$m_2$',  
            ax=ax_a)

### Animation

In [ ]:
%%capture

xmin = -1.2
xmax = -xmin
ymin = -1.2
ymax = -ymin

animated_orbit = plt.figure(figsize=(12,5))
ax_anim = animated_orbit.add_subplot(1,1,1)
ax_anim.set_xlim(xmin,xmax)
ax_anim.set_ylim(ymin,ymax)

m1_anim, = ax_anim.plot(x1, y1, color='blue', lw=1)
m2_anim, = ax_anim.plot(x2, y2, color='red', lw=1)

m1pt_anim, = ax_anim.plot(x1[0],y1[0], 'o', markersize=8, color='blue')
m2pt_anim, = ax_anim.plot(x2[0],y2[0], 'o', markersize=8, color='red')

ax_anim.set_aspect(1)
ax_anim.axis('off')
animated_orbit.tight_layout()

In [ ]:
def animate_orbits(i):
    """
    This funciotn creates the ith animated frame in the objects trajectory.
    """
    
    i_skip = 1 * i
    
    m1pt_anim.set_data(x1[i_skip],y1[i_skip])
    m2pt_anim.set_data(x2[i_skip],y2[i_skip])
    
    return (m1pt_anim , m2pt_anim)

In [ ]:
frame_interval = 10.
frame_num = 1001

anim = animation.FuncAnimation(animated_orbit,
                              animate_orbits,
                              init_func=None,
                              frames = frame_num,
                              interval = frame_interval,
                              blit = True,
                              repeat=False)

In [ ]:
HTML(anim.to_jshtml())

## Leap Frog
Here we will show that using the leapfrog method for solving the differential equation leads to energy conservation in our orbits. We will compare the leapfrog method to the solve_ivp method used to solve the above motion.

In [ ]:
# We will use the same orbit parameters as above but use the leapfrog method here to solve it.

x1Lf, x1_dotLf, x2Lf, x2_dotLf, y1Lf, y1_dotLf, y2Lf, y2_dotLf = o1.solve_ode_Leapfrog(t_pts, z_0)

In [ ]:
# Here we will calculate the change in total energy relative to the intial energy as a function of time.

# solve_ivp
E_tot_pts = o1.energy(t_pts, x1, x1_dot, x2, x2_dot, y1, y1_dot, y2, y2_dot)
E_tot_0 = E_tot_pts[0]
E_tot_rel_pts = np.abs((E_tot_pts - E_tot_0)/E_tot_0)

# Leapfrog
E_tot_pts_Lf = o1.energy(t_pts, x1Lf, x1_dotLf, x2Lf, x2_dotLf, y1Lf, y1_dotLf, y2Lf, y2_dotLf)
E_tot_0_Lf = E_tot_pts_Lf[0]
E_tot_rel_pts_LF = np.abs((E_tot_pts_Lf - E_tot_0_Lf)/E_tot_0_Lf)

In [ ]:
# Plot the energy as a funciton of time

figLf = plt.figure(figsize=(6,6))

overall_title = 'Two Body Orbit:  ' + \
                rf'  $x_1(0) = {x1_0:.1f},$' + \
                rf'  $\dotx_1(0) = {x1_dot_0:.1f},$' + \
                rf'  $y_1(0) = {y1_0:.1f},$' + \
                rf'  $\doty_1(0) = {y1_dot_0:.1f},$' + \
                '\n' + \
                rf'  $x_2(0) = {x2_0:.1f},$' + \
                rf'  $\dotx_2(0) = {x2_dot_0:.1f},$' + \
                rf'  $y_2(0) = {y2_0:.1f},$' + \
                rf'  $\doty_2(0) = {y2_dot_0:.1f},$' + \
                '\n' + \
                rf'  $G = {G:.1f}$' + \
                rf'  $m_1 = {m1:.1f},$' + \
                rf'  $m_2 = {m2:.1f},$'
figLf.suptitle(overall_title, va='baseline')

ax_Lf = figLf.add_subplot(1,1,1)
ax_Lf.semilogy(t_pts, E_tot_rel_pts, 
               color='green', label=r'$\Delta E(t)$ RK23')
ax_Lf.semilogy(t_pts, E_tot_rel_pts_LF, 
               color='red', label=r'$\Delta E(t)$ Leapfrog')
ax_Lf.set_ylim(1.e-9, 1.e1)
ax_Lf.set_xlim(0, 100)
ax_Lf.set_xlabel(r'$t$')
ax_Lf.set_ylabel(r'Energy')
ax_Lf.set_title('Change in energy with time')
ax_Lf.legend(bbox_to_anchor=(1, 1))

figLf.tight_layout()
#figLF.savefig('Leapfrog_energy_test2_t_0.0001.png', dpi=200, bbox_inches='tight')

Here we see that the leapfrog method has energy that is constant with time showing energy conservation, wheras the RK23 method has energy which continues to grow with time showing that it does not conserve energy.

## Orbits with $m_2 >> m_1$

We will compare our computed orbits with the theoretical one given in Taylor 8.6 which studies the case of $m_2 >> m_1$. That section outlines a solution for orbits in polar coordinates given by,

$\begin{align}
  r = \frac{c}{1+\epsilon \cos(\phi)} \hspace{1em} with \hspace{1em} c = \frac{l^2}{G m_1 m_2 \mu} \hspace{1em} \& \hspace{1em} \epsilon = \sqrt{\frac{2l^2 E}{G^2 m_1^2 m_2^2 \mu} + 1}
\end{align}$  

So we can compare our calculated $x$ and $y$ with the $x$ and $y$ predicted from this result

$\begin{align}
  x = \frac{c}{1+\epsilon \cos(\phi)} \cos(\phi) \hspace{1em} \& \hspace{1em} y = \frac{c}{1+\epsilon \cos(\phi)} \sin(\phi)
\end{align}$  




In [ ]:
#Set up the orbit parameters
G2=1. 
m12=1. #Earth
m22=10. #Sun

# Common plotting time (generate the full time then use slices)
t_start = 0.
t_end = 5.
delta_t = 0.001

t_pts = np.arange(t_start, t_end+delta_t, delta_t)  

o2 = Orbit(G=G2, m1=m12, m2=m22)

In [ ]:
#Set these manually
x1_02 = 1.
x1_dot_02 = 0.
y1_02 = 0.
y1_dot_02 = 1.

#Calculated
x2_02, x2_dot_02, y2_02, y2_dot_02 = positionm2(x1_02, x1_dot_02, y1_02, y1_dot_02, m12, m22)

#Calculating the orbits
z_02 = [x1_02, x1_dot_02, x2_02, x2_dot_02, y1_02, y1_dot_02, y2_02, y2_dot_02]
x12, x1_dot2, x22, x2_dot2, y12, y1_dot2, y22, y2_dot2 = o2.solve_ode(t_pts, z_02)

In [ ]:
def PolarParams(x10, x1dot0, x20, x2dot0, y10, y1dot0, y20, y2dot0, m_1, m_2, G):
    """
    Calculates the various parameters needed in the theoretical polar solution for orbits
    """
    l = x10*m_1*y1dot0 - y10*m_1*x1dot0 + x20*m_2*y2dot0 - y20*m_2*x2dot0    #Angular Momentum
    E = 0.5 * m_1 * (x1dot0**2 + y1dot0**2) \
    + 0.5 * m_2 * (x2dot0**2 + y2dot0**2) \
    - G * m_1 * m_2 / np.sqrt((x10 - x20)**2 + (y10 - y20)**2)     #Energy
    mu = m_1*m_2/(m_1+m_2)     #reduced mass
    c = l**2 / (G*m_1*m_2*mu)
    e = np.sqrt((2 * l**2 * E) / (G**2 * m_1**2 * m_2**2 * mu) + 1)     #eccentricity
    return l, E, mu, c, e

In [ ]:
# Calculate the necessary parameters
l, E, mu, c, e = PolarParams(x1_02, x1_dot_02, x2_02, x2_dot_02, y1_02, y1_dot_02, y2_02, y2_dot_02, m12, m22, G2)

# generate the phi points
phi_start = t_start
phi_end = 2*np.pi 
phi_pts = np.linspace(phi_start, phi_end, num=len(t_pts)) 


#calculate the theoretical x and y coordinates
x_theory = [(c / (1. + e*np.cos(phi)))*np.cos(phi) for phi in phi_pts]
y_theory = [(c / (1. + e*np.cos(phi)))*np.sin(phi) for phi in phi_pts]

In [ ]:
#Plotting
fig2 = plt.figure(figsize=(6,6))

overall_title = 'Two Body Orbit:  ' + \
                rf'  $x_1(0) = {x1_02:.1f},$' + \
                rf'  $\dotx_1(0) = {x1_dot_02:.1f},$' + \
                rf'  $y_1(0) = {y1_02:.1f},$' + \
                rf'  $\doty_1(0) = {y1_dot_02:.1f},$' + \
                '\n' + \
                rf'  $x_2(0) = {x2_02:.1f},$' + \
                rf'  $\dotx_2(0) = {x2_dot_02:.1f},$' + \
                rf'  $y_2(0) = {y2_02:.1f},$' + \
                rf'  $\doty_2(0) = {y2_dot_02:.1f},$' + \
                '\n' + \
                rf'  $G = {G2:.1f}$' + \
                rf'  $m_1 = {m12:.1f},$' + \
                rf'  $m_2 = {m22:.1f},$'
fig2.suptitle(overall_title, va='baseline')

ax_a = fig2.add_subplot(1,1,1)
ax_a.set_xlabel(r'$x$')
ax_a.set_ylabel(r'$y$')

start, stop = start_stop_indices(t_pts, t_start, t_end)    
plot_y_vs_x(-x12[start : stop], y12[start : stop],  
            color='red',
            label=r'Computed $m_1$',  
            ax=ax_a) 
plot_y_vs_x(x_theory[start : stop], y_theory[start : stop],  
            color='blue',
            label=r'Theory $m_1$',   
            ax=ax_a)
fig2.tight_layout()

We will repeat a few more times to see the convergence

In [ ]:
#Set up the orbit parameters
G2=1. 
m12a=1. #Earth
m22a=10. #Sun
m12b=1. #Earth
m22b=100. #Sun
m12c=1. #Earth
m22c=1000. #Sun

# Common plotting time (generate the full time then use slices)
t_start = 0.
t_enda = 5.
delta_ta = 0.001
t_endb = 0.5
delta_tb = 0.0001
t_endc = 0.05
delta_tc = 0.00001

t_ptsa = np.arange(t_start, t_enda+delta_ta, delta_ta) 
t_ptsb = np.arange(t_start, t_endb+delta_tb, delta_tb) 
t_ptsc = np.arange(t_start, t_endc+delta_tc, delta_tc) 

o2a = Orbit(G=G2, m1=m12a, m2=m22a)
o2b = Orbit(G=G2, m1=m12b, m2=m22b)
o2c = Orbit(G=G2, m1=m12c, m2=m22c)

In [ ]:
#Set these manually
x1_02 = 1.
x1_dot_02 = 0.
y1_02 = 0.
y1_dot_02 = 1.

#Calculated
x2_02a, x2_dot_02a, y2_02a, y2_dot_02a = positionm2(x1_02, x1_dot_02, y1_02, y1_dot_02, m12a, m22a)
x2_02b, x2_dot_02b, y2_02b, y2_dot_02b = positionm2(x1_02, x1_dot_02, y1_02, y1_dot_02, m12b, m22b)
x2_02c, x2_dot_02c, y2_02c, y2_dot_02c = positionm2(x1_02, x1_dot_02, y1_02, y1_dot_02, m12c, m22c)

#Calculating the orbits
z_02a = [x1_02, x1_dot_02, x2_02a, x2_dot_02a, y1_02, y1_dot_02, y2_02a, y2_dot_02a]
z_02b = [x1_02, x1_dot_02, x2_02b, x2_dot_02b, y1_02, y1_dot_02, y2_02b, y2_dot_02b]
z_02c = [x1_02, x1_dot_02, x2_02c, x2_dot_02c, y1_02, y1_dot_02, y2_02c, y2_dot_02c]
x12a, x1_dot2a, x22a, x2_dot2a, y12a, y1_dot2a, y22a, y2_dot2a = o2a.solve_ode(t_ptsa, z_02a)
x12b, x1_dot2b, x22b, x2_dot2b, y12b, y1_dot2b, y22b, y2_dot2b = o2b.solve_ode(t_ptsb, z_02b)
x12c, x1_dot2c, x22c, x2_dot2c, y12c, y1_dot2c, y22c, y2_dot2c = o2c.solve_ode(t_ptsc, z_02c)

In [ ]:
# Calculate the necessary parameters
la, Ea, mua, ca, ea = PolarParams(x1_02, x1_dot_02, x2_02a, x2_dot_02a, y1_02, y1_dot_02, y2_02a, y2_dot_02a, m12a, m22a, G2)
lb, Eb, mub, cb, eb = PolarParams(x1_02, x1_dot_02, x2_02b, x2_dot_02b, y1_02, y1_dot_02, y2_02b, y2_dot_02b, m12b, m22b, G2)
lc, Ec, muc, cc, ec = PolarParams(x1_02, x1_dot_02, x2_02c, x2_dot_02c, y1_02, y1_dot_02, y2_02c, y2_dot_02c, m12c, m22c, G2)

# generate the phi points
phi_start = t_start
phi_end = 2*np.pi 
phi_ptsa = np.linspace(phi_start, phi_end, num=len(t_ptsa)) 
phi_ptsb = np.linspace(phi_start, phi_end, num=len(t_ptsb))
phi_ptsc = np.linspace(phi_start, phi_end, num=len(t_ptsc))


#calculate the theoretical x and y coordinates
x_theorya = [(ca / (1. + ea*np.cos(phi)))*np.cos(phi) for phi in phi_ptsa]
y_theorya = [(ca / (1. + ea*np.cos(phi)))*np.sin(phi) for phi in phi_ptsa]

x_theoryb = [(cb / (1. + eb*np.cos(phi)))*np.cos(phi) for phi in phi_ptsb]
y_theoryb = [(cb / (1. + eb*np.cos(phi)))*np.sin(phi) for phi in phi_ptsb]

x_theoryc = [(cc / (1. + ec*np.cos(phi)))*np.cos(phi) for phi in phi_ptsc]
y_theoryc = [(cc / (1. + ec*np.cos(phi)))*np.sin(phi) for phi in phi_ptsc]

In [ ]:
#Plotting

fig2 = plt.figure(figsize=(22,6))

overall_title = 'Two Body Orbit:  ' + \
                rf'  $x_1(0) = {x1_02:.1f},$' + \
                rf'  $\dotx_1(0) = {x1_dot_02:.1f},$' + \
                rf'  $y_1(0) = {y1_02:.1f},$' + \
                rf'  $\doty_1(0) = {y1_dot_02:.1f},$'

fig2.suptitle(overall_title, va='baseline')

ax_a = fig2.add_subplot(1,3,1)
ax_a.set_xlabel(r'$x$')
ax_a.set_ylabel(r'$y$')

starta, stopa = start_stop_indices(t_ptsa, t_start, t_enda)    
plot_y_vs_x(-x12a[starta : stopa], y12a[starta : stopa], 
            title=rf'$m_1/m_2={m12a/m22a:.2f}$',
            color='red',
            label=r'Computed $m_1$',  
            ax=ax_a) 
plot_y_vs_x(x_theorya[starta : stopa], y_theorya[starta : stopa], 
            title=rf'$m_1/m_2={m12a/m22a:.2f}$',
            color='blue',
            label=r'Theory $m_1$',  
            ax=ax_a)

ax_b = fig2.add_subplot(1,3,2)
ax_b.set_xlabel(r'$x$')
ax_b.set_ylabel(r'$y$')

startb, stopb = start_stop_indices(t_ptsb, t_start, t_endb)    
plot_y_vs_x(-x12b[startb : stopb], y12b[startb : stopb], 
            title=rf'$m_1/m_2={m12b/m22b:.2f}$',
            color='red',
            label=r'Computed $m_1$', 
            ax=ax_b) 
plot_y_vs_x(x_theoryb[startb : stopb], y_theoryb[startb : stopb],
            title=rf'$m_1/m_2={m12b/m22b:.2f}$',
            color='blue',
            label=r'Theory $m_1$',  
            ax=ax_b)

ax_c = fig2.add_subplot(1,3,3)
ax_c.set_xlabel(r'$x$')
ax_c.set_ylabel(r'$y$')

startc, stopc = start_stop_indices(t_ptsc, t_start, t_endc)    
plot_y_vs_x(-x12c[startc : stopc], y12c[startc : stopc], 
            title=rf'$m_1/m_2={m12c/m22c:.3f}$',
            color='red',
            label=r'Computed $m_1$',  
            ax=ax_c) 
plot_y_vs_x(x_theoryc[startc : stopc], y_theoryc[startc : stopc],
            title=rf'$m_1/m_2={m12c/m22c:.3f}$',
            color='blue',
            label=r'Theory $m_1$',  
            ax=ax_c)
fig2.tight_layout()

Clearly we see that in the limit $m_2 >> m_1$ our orbits approach the theoretically predicted orbit.

## Three Bodies

Next we will look at what happens when we add a third body. We will again confine our attention to three bodies in a two dimensional plane. This is a special case as opposed to the two body case because angular momentum conservation does not force the motion of three bodies to a plane. Regardless, in  two dimensions, we get three equations of motion from Newton's second law,

$\begin{align}
  m_1 \mathbf{\ddot r_1} = \frac{G m_2 m_1}{|\mathbf{r_2 - r_1}|^{3}} (\mathbf{r_2 - r_1}) + \frac{G m_3 m_1}{|\mathbf{r_3 - r_1}|^{3}} (\mathbf{r_3 - r_1}) 
  \hspace{1em} \& \hspace{1em}   
  m_2 \mathbf{\ddot r_2} = \frac{G m_1 m_2}{|\mathbf{r_1 - r_2}|^{3}} (\mathbf{r_1 - r_2}) + \frac{G m_3 m_2}{|\mathbf{r_3 - r_2}|^{3}} (\mathbf{r_3 - r_2}) 
\end{align}$  

$\begin{align}
  m_3 \mathbf{\ddot r_3} = \frac{G m_2 m_3}{|\mathbf{r_2 - r_3}|^{3}} (\mathbf{r_1 - r_2}) + \frac{G m_1 m_3}{|\mathbf{r_1 - r_3}|^{3}} (\mathbf{r_1 - r_3})
\end{align}$  

Again, we introduce, $v_{x1}$ $v_{y1}$ $v_{x2}$ $v_{y2}$ and now $v_{x3}$ $v_{y3}$. This allows us to rewrite our $6$ $2^{nd}$ order differential equations as $12$ $1^{st}$ order equations,

$\begin{align}
  \left(\begin{array}{c} \dot v_{x1}(t) \\ \dot v_{y1}(t) \end{array} \right) 
       = \frac{G m_2}{|\mathbf{r_2 - r_1}|^{3}} \left(\begin{array}{c} x_2(t) - x_1(t) \\ y_2(t) - y_1(t) \end{array} \right) + \frac{G m_3}{|\mathbf{r_3 - r_1}|^{3}} \left(\begin{array}{c} x_3(t) - x_1(t) \\ y_3(t) - y_1(t) \end{array} \right) 
   \hspace{1em} \& \hspace{1em}    
   \left(\begin{array}{c} v_{x1}(t) \\ v_{y1}(t) \end{array} \right) 
   = \left(\begin{array}{c} \dot x_1(t) \\ \dot y_1(t) \end{array} \right)
\end{align}$

$\begin{align}
\left(\begin{array}{c} \dot v_{x2}(t) \\ \dot v_{y2}(t) \end{array} \right) 
       = \frac{G m_1}{|\mathbf{r_1 - r_2}|^{3}} \left(\begin{array}{c} x_1(t) - x_2(t) \\ y_1(t) - y_2(t) \end{array} \right) + \frac{G m_3}{|\mathbf{r_3 - r_2}|^{3}} \left(\begin{array}{c} x_3(t) - x_2(t) \\ y_3(t) - y_2(t) \end{array} \right) 
     \hspace{1em} \& \hspace{1em}
       \left(\begin{array}{c} v_{x2}(t) \\ v_{y2}(t) \end{array} \right) 
       = \left(\begin{array}{c} \dot x_2(t) \\ \dot y_2(t) \end{array} \right)
\end{align}$      





$\begin{align}
\left(\begin{array}{c} \dot v_{x3}(t) \\ \dot v_{y3}(t) \end{array} \right) 
       = \frac{G m_1}{|\mathbf{r_1 - r_3}|^{3}} \left(\begin{array}{c} x_1(t) - x_3(t) \\ y_1(t) - y_3(t) \end{array} \right) + \frac{G m_2}{|\mathbf{r_2 - r_3}|^{3}} \left(\begin{array}{c} x_2(t) - x_3(t) \\ y_2(t) - y_3(t) \end{array} \right) 
     \hspace{1em} \& \hspace{1em}
       \left(\begin{array}{c} v_{x3}(t) \\ v_{y3}(t) \end{array} \right) 
       = \left(\begin{array}{c} \dot x_3(t) \\ \dot y_3(t) \end{array} \right)
\end{align}$  

We will solve for the motion using these three equations

In [ ]:
class ThreeBody():
    """
    Three class solves for the orbit of a three body system with the only interaction being gravity.
     
    Parameters
    ----------
    G : Float
        The gravitational constant
    m1: Float
        The mass of the first planet
    m2: Float 
        The mass of the second planet
    m3: Float 
        The mass of the third planet

    Methods
    -------
    dy_dt(t, z)
        Returns the right side of the differential equation in vector z, 
        given time t and the corresponding value of z.
    
    solve_ode
        Uses solve_ivp to solve the differential equation dy_dt passed to it.
        
    solve_ode_Leapfrog
        Uses the Leapfrog method to solve the differential equation dy_dt passed to it.
        
    energy
        Returns the energy of our system as a function of time.
    """
    def __init__(self, G=1., m1=1., m2=1., m3=1.):
        self.G = G
        self.m1 = m1
        self.m2 = m2
        self.m3 = m3
        
    def dy_dt(self, t, z):
        """
        This function returns the right-hand side of the diffeq: 
        [dphi/dt d^2phi/dt^2]
        
        Parameters
        ----------
        z : float
            A 2-component vector with z[0] = x1(t) and z[1] = dx1(t)/dt
                                      z[2] = x2(t) and z[3] = dx2(t)/dt
                                      z[4] = x3(t) and z[5] = dx3(t)/dt
                                      z[6] = y1(t) and z[7] = dy1(t)/dt
                                      z[8] = y2(t) and z[9] = dy2(t)/dt
                                      z[10] = y3(t) and z[12] = dy3(t)/dt
        t : float
            time 
            
        Returns
        -------
        
        """
        x1, x1_dot, x2, x2_dot, x3, x3_dot, y1, y1_dot, y2, y2_dot, y3, y3_dot = z
        r12 = np.sqrt( (x1 - x2)**2 + (y1 - y2)**2 )
        r13 = np.sqrt( (x1 - x3)**2 + (y1 - y3)**2 )
        r23 = np.sqrt( (x3 - x2)**2 + (y3 - y2)**2 )
        
        return [ \
                x1_dot, (self.m2*self.G*(x2 - x1))/(r12**3) + (self.m3*self.G*(x3 - x1))/(r13**3), \
                x2_dot, (self.m1*self.G*(x1 - x2))/(r12**3) + (self.m3*self.G*(x3 - x2))/(r23**3), \
                x3_dot, (self.m1*self.G*(x1 - x3))/(r13**3) + (self.m2*self.G*(x2 - x3))/(r23**3), \
                y1_dot, (self.m2*self.G*(y2 - y1))/(r12**3) + (self.m3*self.G*(y3 - y1))/(r13**3), \
                y2_dot, (self.m1*self.G*(y1 - y2))/(r12**3) + (self.m3*self.G*(y3 - y2))/(r23**3), \
                y3_dot, (self.m1*self.G*(y1 - y3))/(r13**3) + (self.m2*self.G*(y2 - y3))/(r23**3), \
               ]
    
    def solve_ode(self, t_pts, z_0,
                  method='RK23',
                  abserr=1.0e-8, relerr=1.0e-8):
        """
        Solve the ODE given initial conditions.
        Use solve_ivp with the option of specifying the method.
        Specify smaller abserr and relerr to get more precision.
        """
        
        solution = solve_ivp(self.dy_dt, (t_pts[0], t_pts[-1]), 
                             z_0, t_eval=t_pts, method=method, 
                             atol=abserr, rtol=relerr)
        
        x1, x1_dot, x2, x2_dot, x3, x3_dot, y1, y1_dot, y2, y2_dot, y3, y3_dot = solution.y
        
        return x1, x1_dot, x2, x2_dot, x3, x3_dot, y1, y1_dot, y2, y2_dot, y3, y3_dot
    
    def solve_ode_Leapfrog(self, t_pts, z_0):
        """
        Solve the ODE given initial conditions with the Leapfrog method.
        """
        delta_t = t_pts[1] - t_pts[0]
        
        # initialize the arrays for r, rdot, r_dot_half, phi with zeros
        num_t_pts = len(t_pts)
        
        x1 = np.zeros(num_t_pts)
        x2 = np.zeros(num_t_pts)
        x3 = np.zeros(num_t_pts)
        y1 = np.zeros(num_t_pts)
        y2 = np.zeros(num_t_pts)
        y3 = np.zeros(num_t_pts)
        x1_dot_half = np.zeros(num_t_pts)
        x2_dot_half = np.zeros(num_t_pts)
        x3_dot_half = np.zeros(num_t_pts)
        y1_dot_half = np.zeros(num_t_pts)
        y2_dot_half = np.zeros(num_t_pts)
        y3_dot_half = np.zeros(num_t_pts)
        x1_dot = np.zeros(num_t_pts)
        x2_dot = np.zeros(num_t_pts)
        x3_dot = np.zeros(num_t_pts)
        y1_dot = np.zeros(num_t_pts)
        y2_dot = np.zeros(num_t_pts)
        y3_dot = np.zeros(num_t_pts)
        
        # initial conditions
        x1[0] = z_0[0]
        x1_dot[0] = z_0[1]
        x2[0] = z_0[2]
        x2_dot[0] = z_0[3]
        x3[0] = z_0[4]
        x3_dot[0] = z_0[5]
        y1[0] = z_0[6]
        y1_dot[0] = z_0[7]
        y2[0] = z_0[8]
        y2_dot[0] = z_0[9]
        y3[0] = z_0[10]
        y3_dot[0] = z_0[11]
        
        # step through the differential equation
        for i in np.arange(num_t_pts - 1):
            t = t_pts[i]
            z = [x1[i], x1_dot[i], x2[i], x2_dot[i], x3[i], x3_dot[i], \
                 y1[i], y1_dot[i], y2[i], y2_dot[i], y3[i], y3_dot[i]]
            x1_dot_half[i] = x1_dot[i] + self.dy_dt(t, z)[1] * delta_t/2.
            x2_dot_half[i] = x2_dot[i] + self.dy_dt(t, z)[3] * delta_t/2.
            x3_dot_half[i] = x3_dot[i] + self.dy_dt(t, z)[5] * delta_t/2.
            y1_dot_half[i] = y1_dot[i] + self.dy_dt(t, z)[7] * delta_t/2.
            y2_dot_half[i] = y2_dot[i] + self.dy_dt(t, z)[9] * delta_t/2.
            y3_dot_half[i] = y3_dot[i] + self.dy_dt(t, z)[11] * delta_t/2.
            
            x1[i+1] = x1[i] + x1_dot_half[i]*delta_t
            x2[i+1] = x2[i] + x2_dot_half[i]*delta_t
            x3[i+1] = x3[i] + x3_dot_half[i]*delta_t
            y1[i+1] = y1[i] + y1_dot_half[i]*delta_t
            y2[i+1] = y2[i] + y2_dot_half[i]*delta_t
            y3[i+1] = y3[i] + y3_dot_half[i]*delta_t
            
            z = [x1[i+1], x1_dot[i], x2[i+1], x2_dot[i], x3[i+1], x3_dot[i+1], \
                 y1[i+1], y1_dot[i], y2[i+1], y2_dot[i], y3[i+1], y3_dot[i+1]]
            
            x1_dot[i+1] = x1_dot_half[i] + self.dy_dt(t, z)[1] * delta_t/2.
            x2_dot[i+1] = x2_dot_half[i] + self.dy_dt(t, z)[3] * delta_t/2.
            x3_dot[i+1] = x3_dot_half[i] + self.dy_dt(t, z)[5] * delta_t/2.
            y1_dot[i+1] = y1_dot_half[i] + self.dy_dt(t, z)[7] * delta_t/2.
            y2_dot[i+1] = y2_dot_half[i] + self.dy_dt(t, z)[9] * delta_t/2.
            y3_dot[i+1] = y3_dot_half[i] + self.dy_dt(t, z)[11] * delta_t/2.

        return x1, x1_dot, x2, x2_dot, x3, x3_dot, y1, y1_dot, y2, y2_dot, y3, y3_dot
        
    
    def energy(self, t_pts, x1, x1_dot, x2, x2_dot, x3, x3_dot, y1, y1_dot, y2, y2_dot, y3, y3_dot):
        """Evaluate the energy as a function of time"""
        return (self.m1/2.) * ((x1_dot)**2 + (y1_dot)**2) \
                + (self.m2/2.) * ((x2_dot)**2 + (y2_dot)**2) \
                + (self.m3/2.) * ((x3_dot)**2 + (y3_dot)**2) \
                - (self.G * self.m1 * self.m2) / np.sqrt((x1-x2)**2+(y1-y2)**2) \
                - (self.G * self.m1 * self.m3) / np.sqrt((x1-x3)**2+(y1-y3)**2) \
                - (self.G * self.m2 * self.m3) / np.sqrt((x2-x3)**2+(y2-y3)**2)

### Orbit with Equal Masses

In [ ]:
G=5. 
m1=1.
m2=1.
m3=1.

o3 = ThreeBody(G=G, m1=m1, m2=m2, m3=m3)

Again, in order to have orbits that stay centered at the origin we must impose the condition that the center of mass is at the origin and it's velocity is zero. This time, we allow the inital conditions of two masses to be specified, leaving the third's to be calculated. 

I've written two initial conditions:

1) The first are symmetric about rotations about the origin and give rise to a clean bounded orbit. 

2) The second are the same as the first but I have changed the inital velocity of the first and second masses x-component. We see that the resulting orbits are very different than those from the first set of conditions indicating a sensitivity to inital conditions. While this is a sign of chaos, it is not enough to conclude that this system is chaotic, more analysis would be needed. Also, note that the orbit is no longer bound because the first and third masses move away from the second mass.

In [ ]:
#Set Manually

InitialConditions = 2 # pick either 1 or 2

if InitialConditions == 1:
    x1_0 = np.sqrt(3)/2
    x2_0 = 0.
    x1_dot_0 = 0.5
    x2_dot_0 = -1.
    y1_0 = -0.5
    y2_0 = 1.
    y1_dot_0 = np.sqrt(3)/2
    y2_dot_0 = 0.
    
    # This doesn't need to run as long
    t_start = 0.
    t_end = 5.
    delta_t = 0.001
    t_pts = np.arange(t_start, t_end+delta_t, delta_t) 
    
if InitialConditions == 2:
    x1_0 = np.sqrt(3)/2
    x2_0 = 0.
    x1_dot_0 = 0.50001    #differs by 0.00001
    x2_dot_0 = -1.0001    #differs by 0.0001
    y1_0 = -0.5
    y2_0 = 1.
    y1_dot_0 = np.sqrt(3)/2
    y2_dot_0 = 0.
    
    # This needs to run as longer
    t_start = 0.
    t_end = 50.
    delta_t = 0.001
    t_pts = np.arange(t_start, t_end+delta_t, delta_t)


In [ ]:
def positionm3(x10, x1dot0, y10, y1dot0, x20, x2dot0, y20, y2dot0, m_1, m_2, m_3):
    x30 = -(m_1*x10 + m_2*x20)/m_3
    x3dot0 = -(m_1*x1dot0 + m_2*x2dot0)/m_3
    y30 = -(m_1*y10 + m_2*y20)/m_3
    y3dot0 = -(m_1*y1dot0 + m_2*y2dot0)/m_3
    return x30, x3dot0, y30, y3dot0

In [ ]:
x3_0, x3_dot_0, y3_0, y3_dot_0 = \
    positionm3(x1_0, x1_dot_0, y1_0, y1_dot_0, x2_0, x2_dot_0, y2_0, y2_dot_0, m1, m2, m3)

z_0 = [x1_0, x1_dot_0, x2_0, x2_dot_0, x3_0, x3_dot_0, y1_0, y1_dot_0, y2_0, y2_dot_0, y3_0, y3_dot_0]

x1, x1_dot, x2, x2_dot, x3, x3_dot, y1, y1_dot, y2, y2_dot, y3, y3_dot = o3.solve_ode(t_pts, z_0)

In [ ]:
fig = plt.figure(figsize=(8,8))
ax_a = fig.add_subplot(1,1,1)

overall_title = 'Three Body Orbit:  ' + \
                rf'  $x_1(0) = {x1_0:.1f},$' + \
                rf'  $\dotx_1(0) = {x1_dot_0:.1f},$' + \
                rf'  $y_1(0) = {y1_0:.1f},$' + \
                rf'  $\doty_1(0) = {y1_dot_0:.1f},$' + \
                '\n' + \
                rf'  $x_2(0) = {x2_0:.1f},$' + \
                rf'  $\dotx_2(0) = {x2_dot_0:.1f},$' + \
                rf'  $y_2(0) = {y2_0:.1f},$' + \
                rf'  $\doty_2(0) = {y2_dot_0:.1f},$' + \
                '\n' + \
                rf'  $x_3(0) = {x1_0:.1f},$' + \
                rf'  $\dotx_3(0) = {x3_dot_0:.1f},$' + \
                rf'  $y_3(0) = {y3_0:.1f},$' + \
                rf'  $\doty_3(0) = {y3_dot_0:.1f},$' + \
                '\n' + \
                rf'  $G = {G:.1f}$' + \
                rf'  $m_1 = {m1:.1f},$' + \
                rf'  $m_2 = {m2:.1f},$' + \
                rf'  $m_3 = {m3:.1f},$'
fig.suptitle(overall_title, va='baseline')

ax_a.set_ylim(-3., 3.)
ax_a.set_xlim(-3., 3.)
start, stop = start_stop_indices(t_pts, t_start, t_end)    
plot_y_vs_x(x1[start : stop], y1[start : stop],  
            color='blue',
            label=r'$m_1$',  
            ax=ax_a) 
plot_y_vs_x(x2[start : stop], y2[start : stop],  
            color='red',
            label=r'$m_2$',  
            ax=ax_a)
plot_y_vs_x(x3[start : stop], y3[start : stop],  
            color='green',
            label=r'$m_3$',  
            ax=ax_a)


In [ ]:
%%capture

xmin = -3.
xmax = -xmin
ymin = -3.
ymax = -ymin

animated_three_body = plt.figure(figsize=(12,5))
ax_anim = animated_three_body.add_subplot(1,1,1)
ax_anim.set_xlim(xmin,xmax)
ax_anim.set_ylim(ymin,ymax)

### Comment these out to only see the plants ###
#m1_anim, = ax_anim.plot(x1, y1, color='blue', lw=1)
#m2_anim, = ax_anim.plot(x2, y2, color='red', lw=1)
#m3_anim, = ax_anim.plot(x3, y3, color='green', lw=1)

m1pt_anim, = ax_anim.plot(x1[0],y1[0], 'o', markersize=8, color='blue')
m2pt_anim, = ax_anim.plot(x2[0],y2[0], 'o', markersize=8, color='red')
m3pt_anim, = ax_anim.plot(x3[0],y3[0], 'o', markersize=8, color='green')

ax_anim.set_aspect(1)
ax_anim.axis('off')
animated_three_body.tight_layout()


In [ ]:
def animate_three_bodies(i):
    """
    This funciotn creates the ith animated frame in the objects trajectory.
    """
    if InitialConditions == 1:
        i_skip = 5 * i
        frame_interval = 30.
    if InitialConditions == 2:
        i_skip = 20 * i
        frame_interval = 10.
    
    m1pt_anim.set_data(x1[i_skip],y1[i_skip])
    m2pt_anim.set_data(x2[i_skip],y2[i_skip])
    m3pt_anim.set_data(x3[i_skip],y3[i_skip])
    
    return (m1pt_anim, m2pt_anim, m3pt_anim)

In [ ]:
frame_num = 1001

anim3 = animation.FuncAnimation(animated_three_body,
                              animate_three_bodies,
                              init_func=None,
                              frames = frame_num,
                              interval = frame_interval,
                              blit = True,
                              repeat=False)

In [ ]:
HTML(anim3.to_jshtml())